# **Importing Necessary Libraries**

This imports Pandas (pd) for data manipulation and re (Regular Expressions) for pattern matching. Pandas helps with reading, cleaning, and processing data efficiently. The re module is used for tasks like validating email formats or standardizing text data.

In [13]:
import pandas as pd
import re

# **Loading the Dataset**
The load_data(file_path) function is responsible for reading a CSV file and loading its contents into a Pandas DataFrame. This is done using pd.read_csv(file_path), which allows us to work with the data in a structured tabular format. The function returns the loaded DataFrame so that it can be processed further. If the file path is incorrect or the file is missing, an error will be raised.

In [12]:
def load_data(file_path):
    """Loads customer data from a CSV file."""
    return pd.read_csv(file_path)

# **Identifying Data Issues**
The identify_issues(df) function helps in detecting common data quality problems. It first checks for missing values using df.isnull().sum(), which counts the number of NaN values in each column. Then, it identifies negative ages by counting how many values in the age column are less than zero. For invalid emails, the function uses a regular expression (regex) to filter out incorrectly formatted email addresses. Finally, it counts duplicate rows using df.duplicated().sum(), ensuring that we can remove redundant data later.

In [14]:
def identify_issues(df):
    """Identifies missing values, invalid data, and duplicates."""
    issues = {
        'missing_values': df.isnull().sum(),
        'negative_ages': (df['age'] < 0).sum(),
        'invalid_emails': df[~df['email'].str.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', na=False)].shape[0],
        'duplicates': df.duplicated().sum()
    }
    return issues

# **Handling Missing Values**
The handle_missing_values(df) function fills in missing data to ensure consistency in the dataset. First, it prints the column names for debugging purposes. Then, it replaces missing name values with "Unknown" so that every record has an identifiable name. If country is missing, it is filled with "Not Specified" to avoid empty fields. For purchase_amount, missing values are replaced with the column’s mean (average) to maintain numerical consistency. Instead of using inplace=True, the function assigns the modified DataFrame back to df, which helps prevent chained assignment warnings in Pandas.

In [11]:
def handle_missing_values(df):
    """Fills missing values with appropriate defaults."""
    print("Column Names in DataFrame:", df.columns)

    df['name'] = df['name'].fillna("Unknown")
    df['country'] = df['country'].fillna("Not Specified")
    df['purchase_amount'] = df['purchase_amount'].fillna(df['purchase_amount'].mean())

    return df

# **Fixing Invalid Data**
The fix_invalid_data(df) function corrects incorrect entries. Some records might have negative ages, which do not make sense in a real-world scenario. This is fixed using df['age'].apply(lambda x: abs(x)), which converts negative numbers into positive values. Next, invalid emails are removed from the dataset using regex filtering, ensuring that only correctly formatted email addresses remain. This step improves the accuracy of email-based communication and prevents data entry errors.



In [5]:
# Fix invalid data
def fix_invalid_data(df):
    """Fixes invalid data such as negative ages and invalid emails."""
    df['age'] = df['age'].apply(lambda x: abs(x))
    df = df[df['email'].str.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', na=False)]
    return df

# **Standardizing Data**
The standardize_data(df) function ensures data uniformity by cleaning phone numbers and formatting country names. The phone column is cleaned by removing non-numeric characters using df['phone'].astype(str).str.replace(r'[^0-9]', '', regex=True), which keeps only digits. This is useful for making phone numbers consistent across different formats. The country column is standardized by converting all country names to title case (e.g., "india" → "India") using df['country'].str.title(), improving readability and consistency.



In [6]:
def standardize_data(df):
    """Standardizes phone numbers and country names."""
    df['phone'] = df['phone'].astype(str).str.replace(r'[^0-9]', '', regex=True)
    df['country'] = df['country'].str.title()
    return df

# **Removing Duplicates**
The remove_duplicates(df) function eliminates duplicate rows from the dataset using df.drop_duplicates(). This is important because duplicate records can lead to incorrect analysis, especially when counting unique customers or calculating statistics. By removing duplicates, the dataset becomes cleaner and more reliable.

In [7]:
def remove_duplicates(df):
    """Removes duplicate records."""
    return df.drop_duplicates()

# **Detecting Outliers in Purchase Amount**
The detect_outliers(df) function identifies unusual purchase amounts using the interquartile range (IQR) method. First, it calculates the first quartile (Q1) and third quartile (Q3), then determines the IQR (Q3 - Q1). Any values that fall below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR are considered outliers. Detecting outliers is crucial for identifying potential errors, fraud, or extreme spending behaviors.

In [8]:
def detect_outliers(df):
    """Detects outliers in purchase_amount using the IQR method."""
    q1, q3 = df['purchase_amount'].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df['purchase_amount'] < lower_bound) | (df['purchase_amount'] > upper_bound)]

# **Saving Cleaned Data**
The save_cleaned_data(df, output_file) function saves the cleaned DataFrame to a new CSV file using df.to_csv(output_file, index=False). By setting index=False, it ensures that the DataFrame’s index is not included in the output file. This allows the cleaned dataset to be used for further analysis or machine learning tasks without unnecessary columns.

In [9]:
def save_cleaned_data(df, output_file):
    """Saves the cleaned dataset to a new CSV file."""
    df.to_csv(output_file, index=False)

# **Running the Cleaning Process**
The if __name__ == "__main__": block runs the full cleaning pipeline step by step. It first loads the dataset, identifies issues, and prints them before cleaning. The dataset then goes through missing value handling, invalid data correction, standardization, and duplicate removal. After cleaning, issues are checked again to verify improvements. Finally, the cleaned dataset is saved, and a confirmation message is displayed.

By following this structured approach, the script ensures the dataset is clean, standardized, and free of errors, making it suitable for further analysis or machine learning applications.

In [10]:
# Main Execution
if __name__ == "__main__":
    file_path = "/content/customers.csv"
    output_file = "/content/customers_cleaned.csv"

    df = load_data(file_path)

    print("Issues before cleaning:")
    print(identify_issues(df))

    df = handle_missing_values(df)
    df = fix_invalid_data(df)
    df = standardize_data(df)
    df = remove_duplicates(df)

    print("Issues after cleaning:")
    print(identify_issues(df))

    save_cleaned_data(df, output_file)
    print(f"Cleaned data saved to {output_file}")



Issues before cleaning:
{'missing_values': customer_id         0
name                5
email               0
phone              20
age                 0
country             8
purchase_amount    15
dtype: int64, 'negative_ages': 0, 'invalid_emails': 19, 'duplicates': 0}
Column Names in DataFrame: Index(['customer_id', 'name', 'email', 'phone', 'age', 'country',
       'purchase_amount'],
      dtype='object')
Issues after cleaning:
{'missing_values': customer_id        0
name               0
email              0
phone              0
age                0
country            0
purchase_amount    0
dtype: int64, 'negative_ages': 0, 'invalid_emails': 0, 'duplicates': 0}
Cleaned data saved to /content/customers_cleaned.csv
